# Alternate model
train on weeks 3-8 aggregate statistics with week 9 demand as target; predict on weeks 3-9 aggregate statistics; Route_ID, Sales_Channel_ID, Sales_Depot_ID are masked to NaN if the client-product is present in the training data

best (XGBoost) score: 0.497

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_log_error as rmsle
from sklearn.metrics import root_mean_squared_error

In [3]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
train = pd.read_csv("train.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])
test = pd.read_csv("test.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'id'])

train = train.rename(columns={'Semana': 'Week_num',
                              'Agencia_ID': 'Sales_Depot_ID',
                              'Canal_ID': 'Sales_Channel_ID',
                              'Ruta_SAK': 'Route_ID',
                              'Cliente_ID': 'Client_ID',
                              'Venta_uni_hoy': 'Sales_unit_this_week',
                              'Venta_hoy': 'Sales_this_week',
                              'Dev_uni_proxima': 'Returns_unit_next_week',
                              'Dev_proxima': 'Returns_next_week',
                              'Demanda_uni_equil': 'adjusted_demand',
                              'Producto_ID': 'Product_ID'})

test = test.rename(columns={'Semana': 'Week_num',
                            'Agencia_ID': 'Sales_Depot_ID',
                            'Canal_ID': 'Sales_Channel_ID',
                            'Ruta_SAK': 'Route_ID',
                            'Cliente_ID': 'Client_ID',
                            'Venta_uni_hoy': 'Sales_unit_this_week',
                            'Venta_hoy': 'Sales_this_week',
                            'Dev_uni_proxima': 'Returns_unit_next_week',
                            'Dev_proxima': 'Returns_next_week',
                            'Demanda_uni_equil': 'adjusted_demand',
                            'Producto_ID': 'Product_ID'})



#set a unique id for each sales depot id, sales channel id, route id, client, product combination (thanks Gemini)
combined_df = pd.concat([train,test])
combined_df['ID'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID', 'Product_ID']).ngroup()

#set a combined client ID, consisting of a unique sales depot ID, sales channel ID, route ID, and client ID
combined_df['ccid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID']).ngroup()

#set a combined product ID, consisting of a unique sales depot ID, sales channel ID, route ID, and product ID
combined_df['cpid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Product_ID']).ngroup()

train = combined_df.iloc[:len(train)].copy()
test = combined_df.iloc[len(train):].copy()

del combined_df


train = train.drop(columns=['id'])#, 'Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID'])
train['adjusted_demand'] = train['adjusted_demand'].astype(int)
#train = train.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

test = test.drop(columns=['adjusted_demand', 'Week_num'])#, 'Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Product_ID', 'Client_ID'])
test['id'] = test['id'].astype(int)
#test = test.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)


In [4]:
train['adjusted_demand'] = np.log(train['adjusted_demand']+1)

In [5]:
#create a dataframe of aggregate statistics for each client
testagg = train.sort_values(by=['ccid']).groupby(['ccid'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

client_stats = pd.DataFrame()

client_stats['ccid'] = testagg['ccid']
client_stats['Products'] = testagg['Product_ID']['nunique']
client_stats['client_adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
client_stats['client_adj_dem_median'] = testagg['adjusted_demand']['median'].round(2)
client_stats['client_adj_dem_min'] = testagg['adjusted_demand']['min']
client_stats['client_adj_dem_max'] = testagg['adjusted_demand']['max']

del testagg
#train = train.drop(columns='Product_ID')

#create a dataframe of aggregate statistics for each product
testagg = train.sort_values(by=['cpid']).groupby(['cpid'], as_index=False).agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

product_stats =  pd.DataFrame()

product_stats['cpid'] = testagg['cpid']
product_stats['Clients'] = testagg['Client_ID']['nunique']
product_stats['product_adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
product_stats['product_adj_dem_median'] = testagg['adjusted_demand']['median'].round(2)
product_stats['product_adj_dem_min'] = testagg['adjusted_demand']['min']
product_stats['product_adj_dem_max'] = testagg['adjusted_demand']['max']
#product_stats['median_pct'] = product_stats['adj_dem_median'].rank(pct=True, method='average')

del testagg
#train = train.drop(columns='Client_ID')

In [6]:
#train = pd.merge(train, client_stats, on='ccid', how='left')
test = pd.merge(test, client_stats, on='ccid', how='left')
del client_stats

In [7]:
#train = pd.merge(train, product_stats, on='cpid', how='left')
test = pd.merge(test, product_stats, on='cpid', how='left')
del product_stats

In [8]:
X_train = train[train['Week_num']==9]
train = train[train['Week_num']<9]

In [ ]:
#create a dataframe of aggregate statistics for each client
testagg = train.sort_values(by=['ccid']).groupby(['ccid'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

client_stats = pd.DataFrame()

client_stats['ccid'] = testagg['ccid']
client_stats['Products'] = testagg['Product_ID']['nunique']
client_stats['client_adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
client_stats['client_adj_dem_median'] = testagg['adjusted_demand']['median'].round(2)
client_stats['client_adj_dem_min'] = testagg['adjusted_demand']['min']
client_stats['client_adj_dem_max'] = testagg['adjusted_demand']['max']

del testagg
#train = train.drop(columns='Product_ID')

#create a dataframe of aggregate statistics for each product
testagg = train.sort_values(by=['cpid']).groupby(['cpid'], as_index=False).agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

product_stats =  pd.DataFrame()

product_stats['cpid'] = testagg['cpid']
product_stats['Clients'] = testagg['Client_ID']['nunique']
product_stats['product_adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
product_stats['product_adj_dem_median'] = testagg['adjusted_demand']['median'].round(2)
product_stats['product_adj_dem_min'] = testagg['adjusted_demand']['min']
product_stats['product_adj_dem_max'] = testagg['adjusted_demand']['max']
#product_stats['median_pct'] = product_stats['adj_dem_median'].rank(pct=True, method='average')

del testagg

In [ ]:
X_train = pd.merge(X_train, client_stats, on='ccid', how='left')
del client_stats

In [ ]:
X_train = pd.merge(X_train, product_stats, on='cpid', how='left')
del product_stats

In [12]:
X_train.head()

,Week_num,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,adjusted_demand,ID,ccid,cpid,Products,client_adj_dem_mean,client_adj_dem_median,client_adj_dem_min,client_adj_dem_max,Clients,product_adj_dem_mean,product_adj_dem_median,product_adj_dem_min,product_adj_dem_max
0,9,1110,7,3301,15766,1212,0.693147,2,0,15,42.0,1.72,1.61,0.693147,3.178054,29.0,1.19,1.10,0.693147,2.197225
1,9,1110,7,3301,15766,1238,1.098612,5,0,19,42.0,1.72,1.61,0.693147,3.178054,26.0,1.37,1.39,0.693147,2.302585
2,9,1110,7,3301,15766,1240,1.098612,6,0,20,42.0,1.72,1.61,0.693147,3.178054,32.0,1.57,1.61,0.693147,2.995732
3,9,1110,7,3301,15766,1242,0.693147,7,0,21,42.0,1.72,1.61,0.693147,3.178054,24.0,1.28,1.39,0.693147,2.079442
4,9,1110,7,3301,15766,1250,2.397895,8,0,22,42.0,1.72,1.61,0.693147,3.178054,32.0,1.88,1.95,0.693147,3.401197


In [12]:
X_train = X_train.drop(columns=['Week_num', 'Client_ID', 'Product_ID', 'ID', 'ccid', 'cpid'])

In [13]:
X_train.loc[np.logical_not(np.logical_or(X_train['client_adj_dem_max'].isnull(), X_train['product_adj_dem_max'].isnull())),'Route_ID'] = np.nan
X_train.loc[np.logical_not(np.logical_or(X_train['client_adj_dem_max'].isnull(), X_train['product_adj_dem_max'].isnull())),'Sales_Depot_ID'] = np.nan
X_train.loc[np.logical_not(np.logical_or(X_train['client_adj_dem_max'].isnull(), X_train['product_adj_dem_max'].isnull())),'Sales_Channel_ID'] = np.nan

In [14]:
test.loc[np.logical_not(np.logical_or(test['client_adj_dem_max'].isnull(), test['product_adj_dem_max'].isnull())),'Route_ID'] = np.nan
test.loc[np.logical_not(np.logical_or(test['client_adj_dem_max'].isnull(), test['product_adj_dem_max'].isnull())),'Sales_Depot_ID'] = np.nan
test.loc[np.logical_not(np.logical_or(test['client_adj_dem_max'].isnull(), test['product_adj_dem_max'].isnull())),'Sales_Channel_ID'] = np.nan

In [15]:
X_train['Route_ID'] = X_train['Route_ID'].astype('category')
X_train['Sales_Depot_ID'] = X_train['Sales_Depot_ID'].astype('category')
X_train['Sales_Channel_ID'] = X_train['Sales_Channel_ID'].astype('category')

In [16]:
test['Route_ID'] = test['Route_ID'].astype('category')
test['Sales_Depot_ID'] = test['Sales_Depot_ID'].astype('category')
test['Sales_Channel_ID'] = test['Sales_Channel_ID'].astype('category')

In [18]:
test.head()

,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,id,ID,ccid,cpid,Products,client_adj_dem_mean,client_adj_dem_median,client_adj_dem_min,client_adj_dem_max,Clients,product_adj_dem_mean,product_adj_dem_median,product_adj_dem_min,product_adj_dem_max
0,NaN,NaN,NaN,4639078,35305,0,25973294,2141110,1047658,12.0,1.18,1.10,0.000000,1.791759,66.0,1.63,1.61,0.693147,2.397895
1,NaN,NaN,NaN,4705135,1238,1,23662849,1937430,940421,21.0,1.25,1.10,0.693147,2.397895,61.0,1.01,1.10,0.000000,2.302585
2,NaN,NaN,NaN,4549769,32940,2,21257171,1752098,833109,14.0,1.76,1.70,0.693147,3.713572,48.0,1.15,1.10,0.000000,2.484907
3,NaN,NaN,NaN,4717855,43066,3,5334985,467985,204796,7.0,1.01,0.69,0.693147,1.791759,71.0,0.79,0.69,0.000000,2.772589
4,1219.0,1.0,1130.0,966351,1277,4,4150753,353281,172510,12.0,1.70,1.79,0.693147,3.044522,NaN,NaN,NaN,NaN,NaN


In [17]:
y = X_train['adjusted_demand']
X_train = X_train.drop(columns=['adjusted_demand'])
test = test.drop(columns=['ID', 'ccid', 'cpid'])

In [18]:
test = test.drop(columns = ['id', 'Client_ID', 'Product_ID'])

In [ ]:
#model = xgb.XGBRegressor(n_estimators = 1000, enable_categorical = True)
model = lgb.LGBMRegressor(n_estimators = 300)

In [53]:
model.fit(X_train, y)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.131303 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3019
[LightGBM] [Info] Number of data points in the train set: 10408713, number of used features: 13
[LightGBM] [Info] Start training from score 1.594660


LGBMRegressor(n_estimators=300)

In [ ]:
predictions = model.predict(test)
train_pred = model.predict(X_train)
predictions = np.exp(predictions) - 1
train_pred = np.exp(train_pred) - 1

In [ ]:
y = np.exp(y) - 1

In [59]:
rmsle(y, train_pred)

0.493566148317734

In [60]:
submission = pd.DataFrame()

submission['id'] = np.arange(len(test))
submission['Demanda_uni_equil'] = predictions

In [61]:
submission.head()

,id,Demanda_uni_equil
0,0,3.276222
1,1,1.501366
2,2,2.082781
3,3,0.996319
4,4,2.151131


In [107]:
submission['Demanda_uni_equil']=submission['Demanda_uni_equil'].apply(lambda x: max(x,0))

In [62]:
submission.to_csv("prediction.csv", index=False) #

In [63]:
submission['Demanda_uni_equil'].describe()

count    6.999251e+06
mean     6.125671e+00
std      1.647232e+01
min     -4.681992e-01
25%      2.075361e+00
50%      3.234134e+00
75%      5.664432e+00
max      2.832993e+03
Name: Demanda_uni_equil, dtype: float64